In [44]:
from pyspark.sql import SparkSession

In [45]:
spark = SparkSession.builder.appName('lr_example').getOrCreate()

In [46]:
from pyspark.ml.regression import LinearRegression

In [47]:
data = spark.read.csv('Ecommerce_Customers.csv', header=True, inferSchema=True)

In [48]:
data.printSchema()

root
 |-- Email: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Avatar: string (nullable = true)
 |-- Avg Session Length: double (nullable = true)
 |-- Time on App: double (nullable = true)
 |-- Time on Website: double (nullable = true)
 |-- Length of Membership: double (nullable = true)
 |-- Yearly Amount Spent: double (nullable = true)



In [49]:
data.show()

+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|               Email|             Address|          Avatar|Avg Session Length|       Time on App|   Time on Website|Length of Membership|Yearly Amount Spent|
+--------------------+--------------------+----------------+------------------+------------------+------------------+--------------------+-------------------+
|mstephenson@ferna...|835 Frank TunnelW...|          Violet| 34.49726772511229| 12.65565114916675| 39.57766801952616|  4.0826206329529615|  587.9510539684005|
|   hduke@hotmail.com|4547 Archer Commo...|       DarkGreen| 31.92627202636016|11.109460728682564|37.268958868297744|    2.66403418213262|  392.2049334443264|
|    pallen@yahoo.com|24645 Valerie Uni...|          Bisque|33.000914755642675|11.330278057777512|37.110597442120856|   4.104543202376424| 487.54750486747207|
|riverarebecca@gma...|1414 David Throug...|   

In [50]:
from pyspark.ml.linalg import Vector
from pyspark.ml.feature import VectorAssembler

In [51]:
data.columns

['Email',
 'Address',
 'Avatar',
 'Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership',
 'Yearly Amount Spent']

In [52]:
assembler = VectorAssembler(inputCols=['Avg Session Length',
 'Time on App',
 'Time on Website',
 'Length of Membership'], outputCol='features')

In [53]:
output = assembler.transform(data)

In [54]:
output.head(1)

[Row(Email='mstephenson@fernandez.com', Address='835 Frank TunnelWrightmouth, MI 82180-9605', Avatar='Violet', Avg Session Length=34.49726772511229, Time on App=12.65565114916675, Time on Website=39.57766801952616, Length of Membership=4.0826206329529615, Yearly Amount Spent=587.9510539684005, features=DenseVector([34.4973, 12.6557, 39.5777, 4.0826]))]

In [55]:
final_data = output.select('features', 'Yearly Amount Spent')

In [56]:
final_data.show()

+--------------------+-------------------+
|            features|Yearly Amount Spent|
+--------------------+-------------------+
|[34.4972677251122...|  587.9510539684005|
|[31.9262720263601...|  392.2049334443264|
|[33.0009147556426...| 487.54750486747207|
|[34.3055566297555...|  581.8523440352177|
|[33.3306725236463...|  599.4060920457634|
|[33.8710378793419...|   637.102447915074|
|[32.0215955013870...|  521.5721747578274|
|[32.7391429383803...|  549.9041461052942|
|[33.9877728956856...|  570.2004089636196|
|[31.9365486184489...|  427.1993848953282|
|[33.9925727749537...|  492.6060127179966|
|[33.8793608248049...|  522.3374046069357|
|[29.5324289670579...|  408.6403510726275|
|[33.1903340437226...|  573.4158673313865|
|[32.3879758531538...|  470.4527333009554|
|[30.7377203726281...|  461.7807421962299|
|[32.1253868972878...| 457.84769594494855|
|[32.3388993230671...| 407.70454754954415|
|[32.1878120459321...|  452.3156754800354|
|[32.6178560628234...|   605.061038804892|
+----------

In [57]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

In [58]:
train_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                331|
|   mean|  494.8870162952152|
| stddev|  79.66153087341904|
|    min| 256.67058229005585|
|    max|  744.2218671047146|
+-------+-------------------+



In [59]:
lr = LinearRegression(labelCol='Yearly Amount Spent')

In [60]:
lr_model = lr.fit(train_data)

23/08/06 20:41:01 WARN Instrumentation: [547aea55] regParam is zero, which might cause numerical instability and overfitting.


In [61]:
test_results = lr_model.evaluate(test_data)

In [62]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
|  -3.242870237416298|
|  4.9889269533733795|
| -21.368067492563455|
|  10.945836881105379|
|-0.17898686079331583|
|  3.1871458548811233|
|  0.7916043022210033|
|   3.307317283422833|
| -5.4497540835219525|
|  17.424896663205402|
| -1.4371514585156433|
|  -3.753688203605634|
| -2.3680597904155434|
|  -2.208700477278853|
|  -9.899560870183109|
|  -5.925440471457193|
|  -5.797529312583151|
| -17.777361902124767|
|  -4.128078737106421|
|   5.290478452534217|
+--------------------+
only showing top 20 rows



In [63]:
test_results.rootMeanSquaredError

9.431463271788497

In [64]:
test_results.r2

0.9853446545087268

In [65]:
final_data.describe().show()

+-------+-------------------+
|summary|Yearly Amount Spent|
+-------+-------------------+
|  count|                500|
|   mean|  499.3140382585909|
| stddev|   79.3147815497068|
|    min| 256.67058229005585|
|    max|  765.5184619388373|
+-------+-------------------+



In [66]:
unlabeled_data = test_data.select('features')

In [67]:
unlabeled_data.show()

+--------------------+
|            features|
+--------------------+
|[30.8794843441274...|
|[31.0472221394875...|
|[31.1239743499119...|
|[31.1695067987115...|
|[31.2606468698795...|
|[31.3091926408918...|
|[31.3895854806643...|
|[31.4459724827577...|
|[31.5257524169682...|
|[31.6098395733896...|
|[31.7216523605090...|
|[31.8124825597242...|
|[31.8186165667690...|
|[31.8530748017465...|
|[31.8648325480987...|
|[31.8745516945853...|
|[31.9453957483445...|
|[31.9563005605233...|
|[31.9673209478824...|
|[31.9764800614612...|
+--------------------+
only showing top 20 rows



In [68]:
predictions = lr_model.transform(unlabeled_data)

In [69]:
predictions.show()

+--------------------+------------------+
|            features|        prediction|
+--------------------+------------------+
|[30.8794843441274...|  493.449470222271|
|[31.0472221394875...|  387.508472235648|
|[31.1239743499119...| 508.3151213323292|
|[31.1695067987115...|416.41069392118743|
|[31.2606468698795...| 421.5056181177447|
|[31.3091926408918...| 429.5335719850525|
|[31.3895854806643...| 409.2780067577619|
|[31.4459724827577...|481.56964765170574|
|[31.5257524169682...|449.41538089340384|
|[31.6098395733896...|427.12065298790276|
|[31.7216523605090...| 349.2140780903883|
|[31.8124825597242...|396.56403318740286|
|[31.8186165667690...| 448.7867331605512|
|[31.8530748017465...|461.49382393963083|
|[31.8648325480987...| 449.7908413469968|
|[31.8745516945853...| 398.2106847177247|
|[31.9453957483445...| 662.8174532502351|
|[31.9563005605233...| 564.9032936493236|
|[31.9673209478824...| 449.8779199767587|
|[31.9764800614612...|  325.303967581566|
+--------------------+------------